# data parsing

In [ ]:
import csv
from datetime import datetime
import os
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def get_content(div, value = None):
    if div is None:
        return value
    return div.text.strip()


def convert_date_string_to_datetime(date_str):
    date_format = "%d %B %Y"
    # transform to datetime
    date_obj = datetime.strptime(date_str.strip(), date_format)
    return date_obj


def write_messages_to_csv(messages, csv_file):
    # header
    header = ['Time', 'From', 'Message','Date']

    # write to csv
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        writer.writerow(header)

        for message in messages:
            writer.writerow(message)


def parse_html_and_extract_messages(html_file):
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # parse html
    soup = BeautifulSoup(html_content, 'html.parser')

    # find all div ele class="history" 
    history_div = soup.find('div', class_='history')

    if history_div:
        # find all div ele class="message"
        message_divs = history_div.find_all('div', class_='message')

        first_message = message_divs.pop(0)
        message_date = first_message.text

        message_date = convert_date_string_to_datetime(message_date)

        messages = []
        
        # go thru message_div
        last_name =None
        for message_div in message_divs:
            msg_date = message_div.find('div', class_='date')
            msg_from_name = message_div.find('div', class_='from_name')
            msg_text = message_div.find('div', class_='text')
            
            if msg_date is None:
                continue
                
            name = get_content(msg_from_name)
            
            if name is not None:
                last_name = name
            messages.append([get_content(msg_date), get_content(msg_from_name, last_name), get_content(msg_text), str(message_date)] )

        return  messages


def process_html_files_in_directory(directory_path, output_csv):
    all_messages = []

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.html'):
                file_path = os.path.join(root, file)
                messages = parse_html_and_extract_messages(file_path)
                all_messages.extend(messages)

    # write to csv
    write_messages_to_csv(all_messages, output_csv)


In [ ]:
# process html extract data
directory_path = 'Raw Data'
output_csv = 'all_messages.csv'
process_html_files_in_directory(directory_path, output_csv)


# read 
df = pd.read_csv("all_messages.csv")

for date, group_df in df.groupby(by = 'Date'):
    group_df.to_csv(f'data/{date}.csv', index= False)

# data analysis

## For each day, generate a summary of the content.  ( you can use chatgpt API )  

In [ ]:
# pip install openai
from openai import OpenAI
# OPENAI_API_KEY = "paste key here"
# client = OpenAI(api_key=OPENAI_API_KEY)
import config
import os

In [ ]:
def read_data(filepath):
    with open(filepath) as f:
        lines = f.readlines()
        text = str(lines)
        return text

    
def get_summary_and_save(quotes,filename):
    client = OpenAI(
        api_key=config.api_key,
        base_url=config.base_url
    )

    
    def format_message(role, content):
        return {"role": role, "content": content}


    def get_response(messages):
        completion = client.chat.completions.create(
            model='gpt-4-1106-preview', 
            messages=messages,
        )
        content = completion.choices[0].message.content
        return content


    instructions = f"""
    The following is a group chat record. Write a summary of the chat record of the day based on these contents.
    The summary must be one paragraph. The summary should not exceed 100 words.
    
    Quotes: {quotes}
    """

    message = format_message("system", instructions) # system means high priority
    messages = [message] # ChatGPT API expects any message to be in a list
    response = get_response(messages)


    with open(filename,'w') as f:
        f.write(response)


for root, dirs, files in os.walk('data'):
    for file in files:
        if file.endswith('.csv'):
            print(file)
            file_path = os.path.join(root, file)
            quotes = read_data(file_path)
            filename = file.split(".")[0]
            get_summary_and_save(quotes, f"summary/{filename}_summary.txt")


## How many messages per day 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('all_messages.csv')
df.head()

In [ ]:
df['From'] = df['From'].str.strip()
df.head()

In [ ]:
df['From'].unique()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

messages_per_day = df.groupby(df['Date'].dt.date).size()
messages_per_day

## List the top 5 users that are most active overall

In [ ]:
top_users = df['From'].value_counts().head(5)
top_users